In [2]:
# add chromoscope

import os
import itertools
import pandas as pd

# Example array A
A = ["BOCA-UK-f83fc777-5416-c3e9-e040-11ac0d482c8e_query1", "BOCA-UK-f83fc777-5416-c3e9-e040-11ac0d482c8e_query2", 
     "EX_SPEC_CANCER_VARIANT_PROTOTYPE_sw_1_0_s_1_2_query1", "breast_cancer_circular_s_2_0_oc_query"]

# Folder B (update with your actual path)
folder_B = "../data/chromoscope/imgs"

# Get all filenames in B, stripping extensions
files_B = [os.path.splitext(f)[0] for f in os.listdir(folder_B) if os.path.isfile(os.path.join(folder_B, f))]

# Create all combinations
combinations = list(itertools.product(A, files_B))

# Build DataFrame
df = pd.DataFrame(combinations, columns=["query", "GT_extended"])

# Save to TSV
df.to_csv("query_GT_p2_chromoscope.tsv", sep="\t", index=False)


In [70]:
import os
import re
import pandas as pd
from IPython.display import display_html, HTML

# Define folders
query_folder = '../data/test_suite/imgs'
unified_folder = '../data/unified/imgs'

# Load main DataFrame
df0 = pd.read_csv('GT_absolute.tsv', sep='\t')

# Load A.tsv which contains the list of allowed queries
query_df = pd.read_csv('unmatched_queries_rest5.tsv', sep='\t')  # assuming A is also tab-separated and has a column 'query'

# Load main DataFrame
df0['query'] = df0['query'].str.strip()
query_df['query'] = query_df['query'].str.strip()

# Filter rows in df where 'query' exists in query_df
df = df0[df0['query'].isin(query_df['query'])]

# List all images in the unified folder
all_unified_images = os.listdir(unified_folder)

def find_related_files(groundtruth_name, all_files):
    # Ensure file ends with .png
    if not groundtruth_name.endswith('.png'):
        groundtruth_name += '.png'

    # Determine suffix type: either _cc or _oc
    cc_oc_match = re.search(r'(_viridis(?:_recolor)?)\.png$', groundtruth_name)
    if not cc_oc_match:
        return []

    suffix = cc_oc_match.group()  # like _cc_0.png or _oc.png

    # Get mainname before the last _sw
    sw_split = groundtruth_name.rsplit('_sw', 1)
    if len(sw_split) < 2:
        return []
    mainname = sw_split[0]  # like AREA

    # Find all files with same mainname prefix and suffix (_cc or _oc)
    related = [
        fname for fname in all_files
        if fname.startswith(mainname) and fname.endswith(suffix) and 'HFF' not in fname
    ]
    return related


# Collect data
rows = []
index = 2

for _, row in df.iterrows():
    query_name = row['query']
    groundtruth_name = row['groundtruth'] + '.png'
    
    related_files = find_related_files(groundtruth_name, all_unified_images)
    
    for related_file in related_files:
        rows.append({
            'index': index,
            'query': query_name,
            'query_image': os.path.join(query_folder, query_name + '.png'),
            'groundtruth': row['groundtruth'],
            'groundtruth_image': os.path.join(unified_folder, groundtruth_name),
            'groundtruth_extended': os.path.splitext(related_file)[0],
            'related_image': os.path.join(unified_folder, related_file)
        })
        index += 1

# Create DataFrame
expanded_df = pd.DataFrame(rows)

# Save the DataFrame to a TSV file with two columns: query and related
# expanded_df[['query', 'groundtruth_extended']].to_csv('./query_GT_p11.tsv', sep='\t', index=False)


# Display function
def display_query_gt_related_table(df):
    html = """<table border='1' style='border-collapse: collapse; text-align: center;'>
              <tr><th>#</th>"""
    # html += """<th>Groundtruth</th>"""
    html += """<th>Query</th><th>Related Match</th></tr>"""


    for _, row in df.iterrows():
        html += "<tr>"
        
        html += f"<td>{row['index']}</td>"

        # html += f"""
        #     <td>
        #         <img src="{row['groundtruth_image']}" style="width:250px;"><br>
        #         <div style="padding: 4px; word-break: break-word;">{row['groundtruth']}</div>
        #     </td>
        # """
        
        html += f"""
            <td>
                <img src="{row['query_image']}" style="width:250px;"><br>
                <div style="padding: 4px; word-break: break-word;">{row['query']}</div>
            </td>
        """
            
        html += f"""
            <td>
                <img src="{row['related_image']}" style="width:250px;"><br>
                <div style="padding: 4px; word-break: break-word;">{row['groundtruth_extended']}</div>
            </td>
        """

        html += "</tr>"

    html += "</table>"
    display_html(HTML(html))

# Display the result
display_query_gt_related_table(expanded_df)


#,Query,Related Match
2,EX_SPEC_MATRIX_sw_0_7_s_0_7_oc_viridis_query1,EX_SPEC_MATRIX_sw_0_7_s_0_7_oc_viridis
3,EX_SPEC_MATRIX_sw_0_7_s_0_7_oc_viridis_query2,EX_SPEC_MATRIX_sw_0_7_s_0_7_oc_viridis


In [67]:
import pandas as pd

# Load the two TSV files
df_a = pd.read_csv('unmatched_queries_rest7.tsv', sep='\t')
df_b = pd.read_csv('query_GT_p10.tsv', sep='\t')

# Clean up any extra spaces or invisible characters
df_a['query'] = df_a['query'].str.strip()
df_b['query'] = df_b['query'].str.strip()

# Filter: keep only rows in A where query is NOT in B
filtered_a = df_a[~df_a['query'].isin(df_b['query'])]

# Save or use the result
filtered_a.to_csv('unmatched_queries_rest8.tsv', sep='\t', index=False)


In [18]:
df0

,query,groundtruth
0,AREA_sw_0_7_s_0_7_cc_0_query,AREA_sw_0_7_s_0_7_cc_0
1,AREA_sw_0_7_s_1_0_cc_1_query,AREA_sw_0_7_s_1_0_cc_1
2,AREA_sw_0_7_s_1_2_cc_3_query,AREA_sw_0_7_s_1_2_cc_3
3,AREA_sw_1_0_s_0_7_oc_query,AREA_sw_1_0_s_0_7_oc
4,AREA_sw_1_2_s_0_7_cc_2_query,AREA_sw_1_2_s_0_7_cc_2
...,...,...
199,stratified-line_p_0_sw_1_2_s_1_2_oc_query,stratified-line_p_0_sw_1_2_s_1_2_oc
200,stratified-point_p_0_sw_0_7_s_0_7_cc_0_query,stratified-point_p_0_sw_0_7_s_0_7_cc_0
201,stratified-point_p_0_sw_1_2_s_1_2_oc_query,stratified-point_p_0_sw_1_2_s_1_2_oc
202,viridis-heatmap_p_0_sw_0_7_s_0_7_cc_0_query,viridis-heatmap_p_0_sw_0_7_s_0_7_cc_0


In [76]:
import pandas as pd
import glob

# Path to your TSV files (adjust the pattern as needed)
tsv_files = glob.glob("./components/*.tsv")

all_dfs = []
total_rows = 0

print("File lengths (after header skip and cleaning):")
for file in tsv_files:
    # Read file skipping header row
    df = pd.read_csv(file, sep='\t', skiprows=1, header=None, engine='python', encoding='utf-8', on_bad_lines='skip')
    df = df.dropna(how='all')  # Remove completely empty rows
    print(f"{file}: {len(df)} rows")
    all_dfs.append(df)
    total_rows += len(df)

# Concatenate all DataFrames
combined_df = pd.concat(all_dfs, ignore_index=True)

# Keep and rename only the first two columns
combined_df = combined_df.iloc[:, :2]
combined_df.columns = ['query', 'gt_extended']

# Drop rows with missing query or gt_extended
combined_df = combined_df.dropna(subset=['query', 'gt_extended'])

# Sort alphabetically by 'query'
combined_df = combined_df.sort_values(by='query').reset_index(drop=True)

# Final length
print(f"\nTotal concatenated rows: {len(combined_df)}")

# Save to TSV
combined_df.to_csv("combined_output.tsv", sep='\t', index=False)

File lengths (after header skip and cleaning):
./components/query_GT_p9.tsv: 12 rows
./components/query_GT_p8.tsv: 18 rows
./components/query_GT_p2_chromoscope.tsv: 1924 rows
./components/query_GT_p10.tsv: 4 rows
./components/query_GT_p11.tsv: 2 rows
./components/query_GT_p6.tsv: 6 rows
./components/query_GT_p7.tsv: 14 rows
./components/query_GT_p5.tsv: 7 rows
./components/query_GT_p4.tsv: 133 rows
./components/query_GT_p1.tsv: 1606 rows
./components/query_GT_p3.tsv: 117 rows

Total concatenated rows: 3843


In [ ]:
EX_SPEC_RESPONSIVE_COMPARATIVE_MATRICES